In [ ]:
%load_ext autoreload
%autoreload 2

import os

from datetime import datetime
import matplotlib.pyplot as plt

import numpy as np

from corgisim import scene
from corgisim import instrument

import corgihowfsc

from corgihowfsc.utils.howfsc_initialization import get_args, load_files
from corgihowfsc.sensing.DefaultEstimator import DefaultEstimator
from corgihowfsc.sensing.DefaultProbes import DefaultProbes
from corgihowfsc.utils.contrast_nomalization import CorgiNormalization, EETCNormalization
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage


import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

from howfsc.scripts.gitlframes import sim_gitlframe

import roman_preflight_proper
### Then, run the following command to copy the default prescription file 
roman_preflight_proper.copy_here()

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')
defjacpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\data'
howfscpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\corgihowfsc' #os.path.dirname(os.path.abspath(corgihowfsc.__file__))


# Required setup

In [ ]:
precomp= 'load_all' if defjacpath is not None else 'precomp_all_once'

current_datetime = datetime.now()
folder_name = 'gitl_simulation_' + current_datetime.strftime("%Y-%m-%d_%H%M%S")
fits_name = 'final_frames.fits'
fileout_path = os.path.join(os.path.dirname(os.path.dirname(corgihowfsc.__file__)), 'data', folder_name, fits_name)

dmstartmap_filenames = ['iter_080_dm1.fits', 'iter_080_dm2.fits']

args = get_args(mode='nfov_band1',
                dark_hole='360deg',
                probe_shape='default',
                precomp=precomp,
                num_process=0,
                num_threads=1,
                fileout=fileout_path,
                jacpath=defjacpath,
                dmstartmap_filenames=dmstartmap_filenames)

# Initialize variables etc

otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath

modelpath, cfgfile, jacfile, cstratfile, probefiles, hconffile, n2clistfiles, dmstartmaps = load_files(args, howfscpath)

# cfg
cfg = CoronagraphMode(cfgfile)

# Initialize default probes class
probes = DefaultProbes('default')
dm1_list, dm2_list, dmrel_list, dm10, dm20 = probes.get_dm_probes(cfg, probefiles, dmstartmaps, scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])
hconf = loadyaml(hconffile, custom_exception=TypeError)

cstrat = ControlStrategy(cstratfile)
estimator = DefaultEstimator()

# Imager setup
Here we instantiate the imager function that will call corgisim

In [ ]:
# Get corgisim imager
nrow = 153
ncol = 153
lrow = 0
lcol = 0
crop_corgi = (lrow, lcol, nrow, ncol)

corgi_overrides={}
corgi_overrides['output_dim'] = nrow
corgi_overrides['is_noise_free'] = False

imager = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object
    hconf=hconf,      # Your host config with stellar properties
    backend='corgihowfsc',
    cor=mode,
    corgi_overrides=corgi_overrides
)

# Pick bandpass index (0, 1, 2) for (1A, 1B, 1C) respectively

In [ ]:
sbp_index = 1

# Get camera settings for an image

In [ ]:
contrast = 1e-5

# Set up the EETC
get_cgi_eetc = CGIEETC(mag=hconf['star']['stellar_vmag'],
                   phot='v', # only using V-band magnitudes as a standard
                   spt=hconf['star']['stellar_type'],
                   pointer_path=os.path.join(eetc_path,
                                             hconf['hardware']['pointer']),
)

# Get camera settings

nframes, exptime, gain, snr_out, optflag = \
    get_cgi_eetc.calc_exp_time(
        sequence_name=hconf['hardware']['sequence_list'][sbp_index],
        snr=1,
        scale=contrast,
        scale_bright=contrast,
    )

print(nframes, exptime, gain, snr_out, optflag)

 # Generate coronagraph image
 not directly used but we want the scene later


In [ ]:
# Generate image
im = imager.get_image(dm1_list[0],
                             dm2_list[0],
                             exptime,
                             gain=gain,
                             crop=crop_corgi,
                             lind=1,
                             peakflux=1,
                             cleanrow=1024,
                             cleancol=1024,
                             fixedbp=cstrat.fixedbp,
                             wfe=None)


# Normalization strategy initializations

In [ ]:
# Option 1: EETC
normalization_strategy_1 = EETCNormalization()

# Option 2: Off-axis PSF w/corgisim
normalization_strategy_2 = CorgiNormalization(cfg, cstrat, hconf, cor=args.mode, corgi_overrides=corgi_overrides, separation_lamD=7, exptime_norm=0.01)

In [ ]:

def get_peak_flux_on_axis(dm1_list, nrow, sbp_ind, imager, normalization_strategy_2):
    optics_keywords = {
        'cor_type': 'hlc',
        'use_errors': 2,
        'polaxis': 10,
        'output_dim': nrow,
        'use_dm1': 1,
        'dm1_v': dm1_list[0],
        'use_dm2': 1,
        'dm2_v': dm2_list[0],
        'use_fpm': 0,
        'use_lyot_stop': 1,
        'use_field_stop': 1
    }
    bp = imager.corgisim_manager._get_bandpass_recipe(sbp_ind)
    optics = instrument.CorgiOptics(
        'excam',
        bp,
        optics_keywords=optics_keywords,
        if_quiet=True
    )

    sim_scene = optics.get_host_star_psf(imager.corgisim_manager.base_scene)
    gain_tmp = 1
    emccd_dict = {'em_gain': gain_tmp, 'bias':0, 'cr_rate': 0}
    detector = instrument.CorgiDetector(emccd_dict)

    sim_scene = detector.generate_detector_image(sim_scene, normalization_strategy_2.exptime_norm)
    master_dark = imager.corgisim_manager.generate_master_dark(detector, normalization_strategy_2.exptime_norm, gain_tmp)

    img_nofpm =  (imager.corgisim_manager.k_gain*sim_scene.image_on_detector.data)/gain_tmp - master_dark

    peakflux_3 = np.max(img_nofpm) / normalization_strategy_2.exptime_norm

    return img_nofpm/normalization_strategy_2.exptime_norm, peakflux_3



# Get Peak fluxes for all cases

In [ ]:

_, peakflux_1 = normalization_strategy_1.calc_flux_rate(get_cgi_eetc, hconf, sbp_index, dm1_list[0], dm2_list[0], gain=1)

peak_img_2, peakflux_2 = normalization_strategy_2.calc_flux_rate(get_cgi_eetc, hconf, sbp_index, dm1_list[0], dm2_list[0], gain=1)

peak_img_3, peakflux_3 = get_peak_flux_on_axis(dm1_list, nrow, sbp_index, imager, normalization_strategy_2)

In [ ]:
def get_percent_error(pf1, pf2):
    return 100*(pf2-pf1) / pf1

In [ ]:
pf_2_error = get_percent_error(peakflux_1, peakflux_2)
pf_3_error = get_percent_error(peakflux_1, peakflux_3)
print(f'Peak flux EETC: {peakflux_1:.2f}')
print(f'Peak flux corgisim off-axis: {peakflux_3:.2f}')
print(f'Peak flux corgisim on-axis: {peakflux_1:.2f}')

print(f'EETC vs corgisim off-axis percent error {pf_2_error:.2f}%')
print(f'EETC vs corgisim on-axis percent error {pf_3_error:.2f}%')

In [ ]:
# Plot normalization images

plt.figure(figsize=[12, 4], constrained_layout=True)
plt.subplot(1,2,1)
plt.imshow(np.log10(np.abs(peak_img_2)), cmap='plasma')
# plt.colorbar()
plt.clim([0, 7])
plt.title(f'Corgisim normalized intensity\npeakflux={peakflux_2:.3e}', fontsize=20)


plt.subplot(1,2,2)
plt.imshow(np.log10(np.abs(peak_img_3)), cmap='plasma')
plt.colorbar()
plt.clim([0, 7])
plt.title(f'Corgisim normalized intensity on axis\npeakflux={peakflux_3:.3e}', fontsize=20)


